# Black-Scholes Option Pricing Demo

This notebook demonstrates the Black-Scholes option pricing model with real market data from Yahoo Finance, including calculation of Greeks (sensitivity measures).

In [1]:
import sys
from pathlib import Path

# Add project root to path (works on any machine)
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
from datetime import datetime

from pricing.black_scholes import black_scholes
from pricing.greeks import compute_greeks
from data.fetch_option_chain import fetch_option_chain
from data.cleaning import clean_option_data

## 1. Load & Clean Real Options Data

In [2]:
# Fetch option chain data (nearest expiration)
ticker = 'AAPL'
expiration_date = '2026-01-30'
print(f"Fetching {ticker} option data from Yahoo Finance...")
df_raw = fetch_option_chain(ticker, expiration_date)

print(f"Raw data shape: {df_raw.shape}")
print(f"\nFirst few rows:")
print(df_raw.head())

Fetching AAPL option data from Yahoo Finance...
Raw data shape: (27, 6)

First few rows:
   strike  call_price  put_price    spot  expiration ticker
0   170.0      103.82       0.02  271.01  2026-01-30   AAPL
1   185.0       89.84       0.03  271.01  2026-01-30   AAPL
2   190.0       84.52       0.04  271.01  2026-01-30   AAPL
3   195.0       81.70       0.02  271.01  2026-01-30   AAPL
4   200.0       70.50       0.04  271.01  2026-01-30   AAPL
Raw data shape: (27, 6)

First few rows:
   strike  call_price  put_price    spot  expiration ticker
0   170.0      103.82       0.02  271.01  2026-01-30   AAPL
1   185.0       89.84       0.03  271.01  2026-01-30   AAPL
2   190.0       84.52       0.04  271.01  2026-01-30   AAPL
3   195.0       81.70       0.02  271.01  2026-01-30   AAPL
4   200.0       70.50       0.04  271.01  2026-01-30   AAPL


In [3]:
# Clean the data
df_clean = clean_option_data(df_raw)
print(f"\nCleaned data shape: {df_clean.shape}")
print(f"Removed {len(df_raw) - len(df_clean)} invalid rows")
print(f"\nCleaned data:")
print(df_clean.head(10))


Cleaned data shape: (27, 7)
Removed 0 invalid rows

Cleaned data:
   strike  call_price  put_price    spot expiration ticker         T
0   170.0      103.82       0.02  271.01 2026-01-30   AAPL  0.073973
1   185.0       89.84       0.03  271.01 2026-01-30   AAPL  0.073973
2   190.0       84.52       0.04  271.01 2026-01-30   AAPL  0.073973
3   195.0       81.70       0.02  271.01 2026-01-30   AAPL  0.073973
4   200.0       70.50       0.04  271.01 2026-01-30   AAPL  0.073973
5   205.0       66.35       0.05  271.01 2026-01-30   AAPL  0.073973
6   210.0       60.80       0.09  271.01 2026-01-30   AAPL  0.073973
7   215.0       57.20       0.12  271.01 2026-01-30   AAPL  0.073973
8   220.0       53.47       0.18  271.01 2026-01-30   AAPL  0.073973
9   225.0       48.64       0.22  271.01 2026-01-30   AAPL  0.073973


## 2. Calculate Black-Scholes Prices & Greeks

Use the cleaned data to compute theoretical option prices and Greeks for each strike.

In [ ]:
# Use fixed volatility (could be estimated from market data)
r = 0.05  # Risk-free rate
sigma = 0.20  # Implied volatility (20% annual)

# Calculate BS prices and Greeks for both calls and puts
results = []
for idx, row in df_clean.head(10).iterrows():
    S = row['spot']
    K = row['strike']
    T = row['T']
    
    # Price both calls and puts in a single loop
    for option_type in ['call', 'put']:
        # Get market price based on option type
        market_price = row['call_price'] if option_type == 'call' else row['put_price']
        
        # Calculate BS price and Greeks
        bs_price = black_scholes(S, K, T, r, sigma, option_type=option_type)
        greeks = compute_greeks(S, K, T, r, sigma, option_type=option_type)
        
        results.append({
            'Strike': K,
            'Spot': S,
            'Moneyness': K / S,
            'Type': option_type.capitalize(),
            'Market_Price': market_price,
            'BS_Price': bs_price,
            'Delta': greeks['delta'],
            'Gamma': greeks['gamma'],
            'Vega': greeks['vega'],
            'Theta': greeks['theta'],
        })

results_df = pd.DataFrame(results)
print("Black-Scholes Pricing & Greeks (Calls and Puts):")
print(results_df.to_string(index=False))

# Save all data files (raw, cleaned, and results)
output_dir = project_root / 'data' / 'raw'
output_dir.mkdir(parents=True, exist_ok=True)

# Create timestamp for consistent naming
date_str = pd.to_datetime(expiration_date).strftime("%Y%m%d")

# Define all files to save in a dictionary
files_to_save = {
    f'{ticker}_options_input_{date_str}.csv': df_raw,
    f'{ticker}_options_input_cleaned_{date_str}.csv': df_clean,
    f'{ticker}_options_output_{date_str}.csv': results_df,
}

# Save all files with a single loop
for file_name, dataframe in files_to_save.items():
    output_path = output_dir / file_name
    dataframe.to_csv(output_path, index=False)
    print(f"✓ Saved: {file_name}")

print(f"\nAll data saved to: {output_dir.relative_to(project_root)}")

Black-Scholes Pricing & Greeks (Calls and Puts):
 Strike   Spot  Moneyness Type  Market_Price     BS_Price         Delta        Gamma         Vega         Theta
  170.0 271.01   0.627283 Call        103.82 1.016376e+02  1.000000e+00 1.302931e-18 1.415771e-17 -2.320170e-02
  170.0 271.01   0.627283  Put          0.02 3.648922e-18  0.000000e+00 1.302931e-18 1.415771e-17 -5.161891e-18
  185.0 271.01   0.682632 Call         89.84 8.669298e+01  1.000000e+00 2.769517e-13 3.009370e-12 -2.524891e-02
  185.0 271.01   0.682632  Put          0.03 1.133084e-12 -5.632161e-13 2.769517e-13 3.009370e-12 -1.093519e-12
  190.0 271.01   0.701081 Call         84.52 8.171144e+01  1.000000e+00 8.034099e-12 8.729889e-11 -2.593131e-02
  190.0 271.01   0.701081  Put          0.04 3.763249e-11 -1.749800e-11 8.034099e-12 8.729889e-11 -3.167816e-11
  195.0 271.01   0.719531 Call         81.70 7.672990e+01  1.000000e+00 1.694823e-10 1.841603e-09 -2.661371e-02
  195.0 271.01   0.719531  Put          0.02 9.137941e-

## 3. Summary

The Black-Scholes model provides:
- **Theoretical option prices** for comparison with market prices
- **Greeks** to measure exposure:
  - **Delta**: Change in option price per $1 spot move
  - **Gamma**: Delta sensitivity to spot moves
  - **Vega**: Sensitivity to volatility changes (per 1%)
  - **Theta**: Daily time decay

In a real trading system, we would:
1. Estimate volatility from market prices (implied vol)
2. Use market data for risk-free rate
3. Monitor Greeks for portfolio hedging